# Artwork sequence prediction

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
#CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/multivariate'
#CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/univariate'
CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration/univariate_predict_multiple'

In [3]:
window_index = 3

museum_sequence_path = {
    'x_train' : os.path.join(CONFIG_PATH, 'X_train.csv'),
    'x_test' : os.path.join(CONFIG_PATH, 'X_test.csv'),
    'x_train_matrix' : os.path.join(CONFIG_PATH, 'X_train_matrix.npy'),
    'x_test_matrix' : os.path.join(CONFIG_PATH, 'X_test_matrix.npy'),
    'weights_folder' : os.path.join(CONFIG_PATH, 'config_'+str(window_index)+'/trained_model_weights')
}
museum_sequence_path

{'weights_folder': '/root/work/artwork_sequence/train_test_configuration/univariate_predict_multiple/config_3/trained_model_weights',
 'x_test': '/root/work/artwork_sequence/train_test_configuration/univariate_predict_multiple/X_test.csv',
 'x_test_matrix': '/root/work/artwork_sequence/train_test_configuration/univariate_predict_multiple/X_test_matrix.npy',
 'x_train': '/root/work/artwork_sequence/train_test_configuration/univariate_predict_multiple/X_train.csv',
 'x_train_matrix': '/root/work/artwork_sequence/train_test_configuration/univariate_predict_multiple/X_train_matrix.npy'}

## Load data

In [4]:
df_x_train = pd.read_csv(museum_sequence_path['x_train'], index_col=0)
df_x_test = pd.read_csv(museum_sequence_path['x_test'], index_col=0)
x_train_matrix = np.load(museum_sequence_path['x_train_matrix'])
x_test_matrix = np.load(museum_sequence_path['x_test_matrix'])
df_x_train.head()

,tour_path
20,/root/work/datasets/artwork_sequence/rijksmuse...
7,/root/work/datasets/artwork_sequence/rijksmuse...
40,/root/work/datasets/artwork_sequence/prado_cra...
0,/root/work/datasets/artwork_sequence/rijksmuse...
23,/root/work/datasets/artwork_sequence/prado_cra...


In [5]:
x_train_matrix.shape

(637, 300)

## Reset Tensorflow session

In [6]:
tf.keras.backend.clear_session()

## Config data to fit with the model input

Because the **Prediction feature model** split the data into training and validation dataset, it is necessary to give all the data in only one block

**Define timeline**

In [7]:
time = np.arange(x_train_matrix.shape[0] + x_test_matrix.shape[0])
time.shape

(859,)

**Define configuration to deal with the windowed dataset**

In [8]:
split_time = x_train_matrix.shape[0]

X = np.concatenate((x_train_matrix, x_test_matrix))

#length of the history
window_size = window_index

#Number of artwork's feature
n_features = X.shape[1]

#Number of feature to take into account
n_influence_features=10

batch_size = 128
shuffle_buffer_size = 300

## Create models

**Define model**

In [9]:
from Sequence_prediction_factory import Sequence_prediction_multivariate, Sequence_prediction_univariate

In [10]:
model_univariate = Sequence_prediction_univariate(
    X=X, 
    shuffle_buffer_size=shuffle_buffer_size, 
    split_time=split_time, 
    train_batch_size=batch_size, 
    val_batch_size=batch_size, 
    window_size=window_size)

In [11]:
model_multivariate = Sequence_prediction_multivariate(
    X=X, 
    shuffle_buffer_size=shuffle_buffer_size, 
    split_time=split_time, 
    train_batch_size=batch_size, 
    val_batch_size=batch_size, 
    window_size=window_size, 
    n_influence_features=n_influence_features)

In [12]:
model_prediction = model_univariate

In [13]:
from IPython.display import clear_output
import time

models = []
start_time = time.time()
for i in range(n_features):
    clear_output(wait=True)
    print("---------- Feature %s -------------" % (i))
    
    model_prediction.set_index(i)
    model = model_prediction.get_model()
    model.define_model(conv_filter=16, lstm_filter=32, dense_filter=16, prediction_length=10)
    model.train_model(epochs=20, lr=1e-6)
    models.append(model)
    
    #Save weights
    model.save_weights(museum_sequence_path)
    
print("--- %s seconds ---" % (time.time() - start_time))

---------- Feature 299 -------------
Epoch 1/20
5/5 [==============================] - 7s 1s/step - loss: 1.1573 - mae: 1.5329 - val_loss: 0.0000e+00 - val_mae: 0.0000e+00
Epoch 2/20
5/5 [==============================] - 0s 37ms/step - loss: 0.8817 - mae: 1.2262 - val_loss: 0.9360 - val_mae: 1.2873
Epoch 3/20
5/5 [==============================] - 0s 38ms/step - loss: 0.6423 - mae: 0.9903 - val_loss: 0.7193 - val_mae: 1.0852
Epoch 4/20
5/5 [==============================] - 0s 37ms/step - loss: 0.5169 - mae: 0.8698 - val_loss: 0.6005 - val_mae: 0.9684
Epoch 5/20
5/5 [==============================] - 0s 40ms/step - loss: 0.4501 - mae: 0.7975 - val_loss: 0.5268 - val_mae: 0.8817
Epoch 6/20
5/5 [==============================] - 0s 39ms/step - loss: 0.3947 - mae: 0.7292 - val_loss: 0.4686 - val_mae: 0.8130
Epoch 7/20
5/5 [==============================] - 0s 40ms/step - loss: 0.3423 - mae: 0.6666 - val_loss: 0.4208 - val_mae: 0.7554
Epoch 8/20
5/5 [==============================] - 0s 3

In [14]:
len(models)

300

In [15]:
models[0].model.summary()

AttributeError: 'Prediction_model_feature' object has no attribute 'model'